- Translation Invariance. Não importa onde o gato esta na imagem, só importa que é um gato na imagem. Para conseguir usar este conceito usamos **weight sharing**

- Weight Sharing. Qndo 2 inputs tem o mesmo tipo de informação, seus pesos podem ser compartilhados e aprimora estes pesos juntos.

## Convol. Networks (CNN)

- ConNets compartilham seus pesos pelo espaço (alt, larg, profund.)

The first step for a CNN is to break up the image into smaller pieces. We do this by selecting a width and height that defines a filter.

The filter looks at small pieces, or patches, of the image. These patches are the same size as the filter.

- **filter = patch = kernels**
- **qdt filtros = filter depth**

![](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/58377d67_vlcsnap-2016-11-24-15h52m47s438/vlcsnap-2016-11-24-15h52m47s438.png)

What's important here is that we are grouping together adjacent pixels and treating them as a collective.

It's common to have more than one filter. Different filters pick up different qualities of a patch. For example, one filter might look for a particular color, while another might look for a kind of object of a specific shape. The amount of filters in a convolutional layer is called the filter depth.

### Parameter Sharing

![](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/58377f77_vlcsnap-2016-11-24-16h01m35s262/vlcsnap-2016-11-24-16h01m35s262.png)

The weights, w, are shared across patches for a given layer in a CNN to detect the cat above regardless of where in the image it is located.

### Dimensionality

Com as informações passadas podemos calcular o numero de neuronios paraca cada camada em uma CNN:

- Nossa camada de entrada tem uma largura de W e uma altura de H
- Nossa camada convolucional tem um tamanho de filtro F
- Nós temos um passo (stride) de S
- Um preenchimento (padding) de P
- E o número de filtros K

A seguinte fórmula nos dá a largura da próxima camada: $W_{out} = (W - F + 2P) / S + 1$

A altura de saída seria $H_{out} = (H - F + 2P) / S + 1$

E a profundidade de saída seria igual ao número de filtros $D_{out} = K$

O volume de saída seria $W_{out} * H_{out} * D_{out}$

### Quiz 1

H = height, W = width, D = depth

- We have an input of shape 32x32x3 (HxWxD)
- 20 filters of shape 8x8x3 (HxWxD)
- A stride of 2 for both the height and width (S)
- With padding of size 1 (P)

In [3]:
input_height = 32
filter_height = 8
P = 1
S = 2

new_height = (input_height - filter_height + 2 * P)/S + 1
print(new_height)

14.0


In [4]:
input_width = 32
filter_width = 8
P = 1
S = 2

new_width = (input_width - filter_width + 2 * P)/S + 1
print(new_width)

14.0


In [6]:
depth = 20

output = '%s x %s x %s' % (new_height, new_width, depth)
print(output)

14.0 x 14.0 x 20


Usando TensorFlow ficaria:

In [9]:
import tensorflow as tf

input = tf.placeholder(tf.float32, (None, 32, 32, 3))
filter_weights = tf.Variable(tf.truncated_normal((8, 8, 3, 20))) # (height, width, input_depth, output_depth)
filter_bias = tf.Variable(tf.zeros(20))
strides = [1, 2, 2, 1] # (batch, height, width, depth)
padding = 'SAME'
conv = tf.nn.conv2d(input, filter_weights, strides, padding) + filter_bias

Porem, o TensorFlow usa um algoritmo diferente, em suma as equações abaixo representam as opções 'SAME' e 'PADDING':

SAME Padding, o output height e width são calculados como:

In [ ]:
out_height = ceil(float(in_height) / float(strides1))

out_width = ceil(float(in_width) / float(strides[2]))

VALID Padding, o output height e width são calculados como:

In [ ]:
out_height = ceil(float(in_height - filter_height + 1) / float(strides1))

out_width = ceil(float(in_width - filter_width + 1) / float(strides[2]))

### Quiz 2

Com o output layer do Quiz 1 calcule a quantidade de parametros esta CNN teria sem compartilhamento de parametros. (Sem o compartilhamento de parâmetros, cada neurônio na camada de saída deve se conectar a cada neurônio no filtro. Além disso, cada neurônio na camada de saída também deve se conectar a um único neurônio de bias)

In [15]:
out_put_layer_neurons = 14*14*20

filter_neuros = 8*8*3

bias_neurons = out_put_layer_neurons

total_parameters = (out_put_layer_neurons * filter_neuros) + bias_neurons

print(total_parameters)

756560


### Quiz 3

Como o mesmos parametros do Quiz 1, calcular o total de parametros **com** compartilhamento de parametros. (Com o compartilhamento de parâmetros, cada neurônio em um canal de saída compartilha seus pesos com cada outro neurônio nesse canal. Assim, o número de parâmetros é igual ao número de neurônios no filtro, mais um neurônio de polarização, todos multiplicados pelo número de canais na camada de saída.)

In [16]:
output_channels = 20

filter_neuros = 8*8*3

total_parameters = output_channels * (filter_neuros + 1)

print(total_parameters)

3860


### Exemplo em TensorFlow 

In [17]:
import tensorflow as tf

# Output depth
k_output = 64

# Image Properties
image_width = 10
image_height = 10
color_channels = 3

# Convolution filter
filter_size_width = 5
filter_size_height = 5

# Input/Image
input = tf.placeholder(
    tf.float32,
    shape=[None, image_height, image_width, color_channels])

# Weight and bias
weight = tf.Variable(tf.truncated_normal(
    [filter_size_height, filter_size_width, color_channels, k_output]))
bias = tf.Variable(tf.zeros(k_output))

# Apply Convolution
conv_layer = tf.nn.conv2d(input, weight, strides=[1, 2, 2, 1], padding='SAME')
# Add bias
conv_layer = tf.nn.bias_add(conv_layer, bias)
# Apply activation function
conv_layer = tf.nn.relu(conv_layer)

O código acima usa a função tf.nn.conv2d() para calcular a convolução com **peso** como filtro e **[1, 2, 2, 1]** para os passos. TensorFlow usa um *stride* para cada dimensão de **entrada**, *[batch, input_height, input_width, input_channels]*. Geralmente, sempre vamos definir o *stride* para o *batch* e *input_channels* (ou seja, o primeiro eo quarto elemento na matriz strides) para ser 1.

Você se concentrará em alterar *input_height* e *input_width* enquanto define *batch* e *input_channels* como 1. Os strides *input_height* e *input_width* são para passar o filtro pela *entrada*. Este código de exemplo usa um stride de 2 com filtro 5x5 sobre *entrada*.

## CovNet Avançado

![](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581a58be_convolution-schematic/convolution-schematic.gif)

Abaixo são listados alguns métodos para aprimorar uma CNN.

- Pooling - Usado para reduzir as dimesões espaciais (down-sampling) dos dados de input, enquanto contribui para evitar o overfitting. Geralmente aplicado entre camadas "convolution".

Recentemente, pooling layers perderam a preferencia por algumas razões:

    - Os datasets recentes são tão grandes e complexos que o que mais preocupa é o underfittig; 
    - Dropout é um regularizador melhor
    - Polling resulta em perda de dados.

In [ ]:
conv_layer = tf.nn.conv2d(input, weight, strides=[1, 2, 2, 1], padding='SAME')
conv_layer = tf.nn.bias_add(conv_layer, bias)
conv_layer = tf.nn.relu(conv_layer)
# Apply Max Pooling
conv_layer = tf.nn.max_pool(
    conv_layer,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='SAME')

- 1x1 convolutions - Tbm usado para reduzir dimensões especiais. Atua na redução das dimesões do filtro.

- Inception - Em cada camada da CNN podemos escolher entre vária opções a aplicar, 1x1 conv, 3x3 conv, 5x5 conv, pooling etc. Qual escolher? Inception module usa todas as opções e concatena suas saídas.

### Quiz 4

Configurar a CNN para, a partir de um input (1,4,4,1), dar um output (1,2,2,3)

In [2]:
"""
Setup the strides, padding and filter weight/bias such that
the output shape is (1, 2, 2, 3).
"""
import tensorflow as tf
import numpy as np

# `tf.nn.conv2d` requires the input be 4D (batch_size, height, width, depth)
# (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)


def conv2d(input):
    # Filter (weights and bias)
    # The shape of the filter weight is (height, width, input_depth, output_depth)
    # The shape of the filter bias is (output_depth,)
    # TODO: Define the filter weights `F_W` and filter bias `F_b`.
    # NOTE: Remember to wrap them in `tf.Variable`, they are trainable parameters after all.
    F_W = tf.Variable(tf.random_normal([4, 4, 1, 3]))
    F_b = tf.Variable(tf.zeros(3))
    # TODO: Set the stride for each dimension (batch_size, height, width, depth)
    strides = [1, 2, 2, 1]
    # TODO: set the padding, either 'VALID' or 'SAME'.
    padding = 'SAME'
    # https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#conv2d
    # `tf.nn.conv2d` does not include the bias computation so we have to add it ourselves after.
    return tf.nn.conv2d(input, F_W, strides, padding) + F_b

print(conv2d(X))

Tensor("add_1:0", shape=(1, 2, 2, 3), dtype=float32)


### Quiz 5

No exercício abaixo, você será solicitado a configurar as dimensões dos filtros de agrupamento, passadas, bem como o preenchimento apropriado. Você deve consultar a documentação do TensorFlow para tf.nn.max_pool (). Padding funciona da mesma forma que faz para uma convolução

In [3]:
"""
Set the values to `strides` and `ksize` such that
the output shape after pooling is (1, 2, 2, 1).
"""
import tensorflow as tf
import numpy as np

# `tf.nn.max_pool` requires the input be 4D (batch_size, height, width, depth)
# (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)

def maxpool(input):
    # TODO: Set the ksize (filter size) for each dimension (batch_size, height, width, depth)
    ksize = [1, 2, 2, 1]
    # TODO: Set the stride for each dimension (batch_size, height, width, depth)
    strides = [1, 2, 2, 1]
    # TODO: set the padding, either 'VALID' or 'SAME'.
    padding = 'SAME'
    # https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#max_pool
    return tf.nn.max_pool(input, ksize, strides, padding)
    
print(maxpool(X))

Tensor("MaxPool:0", shape=(1, 2, 2, 1), dtype=float32)


## Fontes adicionais

These are the resources we recommend in particular:

- Andrej Karpathy's [CS231n Stanford course](http://cs231n.github.io/) on Convolutional Neural Networks.
- Michael Nielsen's [free book](http://neuralnetworksanddeeplearning.com/) on Deep Learning.
- Goodfellow, Bengio, and Courville's more advanced [free book](http://deeplearningbook.org/) on Deep Learning.